In [ ]:
pip3 install python-chess==0.28.1

In [ ]:
conda install pandas

In [ ]:
from IPython.core.display import HTML, display
HTML("""
<style>
svg {
    width:40% !important;
    height:40% !important;
}

.container { 
    width:100% !important;
}
</style>
""")

In [ ]:
OPENING_MAX_FULLMOVE_NUM = 6
MAX_BOARD_VALUE = float("inf")
MAX_DEPTH = 8
TIME_LIMIT = 45

def get_move(board):        
        opening_book = import_opening_book(OPENING_BOOK_LOC)
        
        if board.fullmove_number <= OPENING_MAX_FULLMOVE_NUM:
            move = get_opening_move(board, opening_book)
            if not move is None:
                return move
        

        return iterative_deepening(board, MAX_DEPTH, evaluate_board)

In [ ]:
OPENING_BOOK_LOC = "./res/polyglot/Performance.bin"

In [ ]:
def import_opening_book(book_location):
        if os.path.isfile(book_location):
            return chess.polyglot.open_reader(book_location)
        else:
            raise FileNotFoundError(
                errno.ENOENT, os.strerror(errno.ENOENT), book_location)

In [ ]:
def get_opening_move(board, opening_book):
        '''
        get the current board and return move, as string, for this situation
        '''
        if not (opening_book is None):
            try:
                main_entry = opening_book.weighted_choice(board)
                move = main_entry.move
                opening_book.close()
                return move
            except IndexError:
                return None
        else:
            return None

In [ ]:
def iterative_deepening(board, max_depth, evaluation_func):
    depth = 1

    start_time = int(time.time())
    end_time = start_time + TIME_LIMIT
    current_time = start_time

    player = bool(board.turn)
    best_possible_result = get_best_possible_result(board, player)

    legal_moves = list(board.legal_moves)
    while current_time < end_time and depth <= max_depth:
        move_val_dict = {}

        best_value = float('-inf')
        best_move = legal_moves[0]

        for move in legal_moves:
            board.push(move)
            value = min_value(board, player, float('-inf'), float('inf'), depth - 1, end_time, evaluation_func, best_possible_result)
            board.pop()
            if value is False:
                value = float('-inf')
            move_val_dict[move] = value
            if value == MAX_BOARD_VALUE:
                return move
            if value > best_value:
                best_value = value
                best_move = move

        legal_moves.sort(key=move_val_dict.get, reverse=True)
        depth += 1
        current_time = int(time.time())

    return best_move

In [ ]:
def get_best_possible_result(board, player):
    if player and board.has_insufficient_material(chess.WHITE):
        return 0
    if not player and board.has_insufficient_material(chess.BLACK):
        return 0
    if player and not board.has_insufficient_material(chess.WHITE):
        return 1
    if not player and not board.has_insufficient_material(chess.BLACK):
        return -1

In [ ]:
def min_value(board, player, alpha, beta, depth, time_limit, evaluation_func, best_possible_result):
    v = float('inf')
    
    if board.is_game_over() or depth == 0:
        return evaluation_func(board, player, best_possible_result)
    if int(time.time()) >= time_limit:
        return False

    for move in board.legal_moves:
        board.push(move)
        deeper_val = max_value(board, player, alpha, beta, depth -1, time_limit, evaluation_func, best_possible_result)
        board.pop()
        if deeper_val is False:
            return False            
        v = min(v, deeper_val)  

        if v <= alpha:
            return v
        beta = min(beta, v)
    return v

In [ ]:
def max_value(board, player, alpha, beta, depth, time_limit, evaluation_func, best_possible_result):
    v = float('-inf')

    if board.is_game_over() or depth == 0:
        return evaluation_func(board, player, best_possible_result)
    if int(time.time()) >= time_limit:
        return False

    for move in board.legal_moves:
        board.push(move)
        deeper_val = min_value(board, player, alpha, beta, depth -1, time_limit, evaluation_func, best_possible_result)
        board.pop()
        if deeper_val is False:
            return False
        v = max(v, deeper_val)

        if v >= beta:
            return v
        alpha = max(alpha, v)
    return v

In [ ]:
def evaluate_board(board, player, best_possible_result):
    player_color = chess.WHITE if player else chess.BLACK

    if board.is_game_over():
        result = get_board_result(board)
        if result is best_possible_result:
            return MAX_BOARD_VALUE
        if result is best_possible_result * -1:
            return -1 * MAX_BOARD_VALUE
    
    return get_board_value(board, player)

In [ ]:
PAWN_VALUE = 1
ROOK_VALUE = 5
KNIGHT_VALUE = 3
BISHOP_VALUE = 3
QUEEN_VALUE = 9
KING_VALUE = 15

def assign_piece_value(piece_type, count_king=True):
    return {
        1: PAWN_VALUE,
        2: KNIGHT_VALUE,
        3: BISHOP_VALUE,
        4: ROOK_VALUE,
        5: QUEEN_VALUE,
        6: KING_VALUE if count_king else 0
    }.get(piece_type, 0)


def get_value_by_color(board, color, count_king=True):
    pieces_value = map(
        lambda piece_type: len(board.pieces(piece_type, color)) * assign_piece_value(piece_type, count_king), chess.PIECE_TYPES)
    return sum(pieces_value)


def get_board_value(board, color, count_king=True):
    white_value = get_value_by_color(board, chess.WHITE, count_king)
    black_value = get_value_by_color(board, chess.BLACK, count_king)

    return white_value - black_value if color is chess.WHITE else black_value - white_value


In [ ]:
from IPython.display import SVG, display

def print_board_svg(board):
    display(SVG(chess.svg.board(board=board)))

In [ ]:
def get_board_result(board):
    if board.is_variant_loss():
        return -1 if board.turn == chess.WHITE else 1
    elif board.is_variant_win():
        return 1 if board.turn == chess.WHITE else -1
    elif board.is_variant_draw():
        return 0

    if board.is_checkmate():
        return -1 if board.turn == chess.WHITE else 1
    if board.can_claim_draw():
        return 0
    if board.is_seventyfive_moves() or board.is_fivefold_repetition():
        return 0
    if board.is_insufficient_material():
        return 0
    if not any(board.generate_legal_moves()):
        return 0
    
    return 0

In [ ]:
import pandas as pd
import chess
import chess.svg
import chess.polyglot
import os
import numpy as np
import time

board = chess.Board()
while not board.is_game_over():
    print_board_svg(board)
    if board.turn:
        legal_moves = list(map(board.uci, board.legal_moves))
        move = None
        while move not in legal_moves:
            print("Legal moves:")
            print(legal_moves)
            move = input("Please enter your move: ")
        move = chess.Move.from_uci(move)
    else:
        move = get_move(board)

    board.push(move)

result = get_board_result(board)
if result is 1:
    print("{} (White) has won".format(players[0].name))
elif result is -1:
    print("{} (Black) has won".format(players[1].name))
else:
    print("Draw")
    